In [1]:
'''
Import packages necessary to run script
'''
# def import_packages():
# import tkinter as tk
# from tkinter import filedialog, simpledialog, messagebox
import math
# from statistics import mean, stdev
import numpy as np
import pandas as pd
# import os
# %matplotlib inline
# from datetime import datetime
# import matplotlib.pyplot as plt
# import scipy.stats as sps
# import sys
# import scipy.optimize as spo
# return print('packages imported')
# import_packages()

In [2]:
'''
Fetch working directory, and set modules directory
'''
import os, sys
cwd = os.getcwd()
modules_path = cwd + '\Modules\\'
sys.path.insert(1, modules_path)
print(f"Added {modules_path} as modules folder")

Added C:\Users\Matt\OneDrive - University of Toronto\Scripts\Revised Script - Feb 2022\Modules\ as modules folder


In [3]:
'''
Set up input and output folders and scan settings
'''
def initial_setup(addfolderswithdate, getinput_type):
    import InitialSetup as InitialSetup
    input_output_folders = InitialSetup.get_folders(withdate=addfolderswithdate)
    settings = InitialSetup.get_data_type(get_input=True)
    return (input_output_folders, settings)

addfolderswithdate, getinput_type = False, True #change this to make new folder with date
initial_setup_output = initial_setup(addfolderswithdate, getinput_type)
file_path_input = initial_setup_output[0][0]
file_path_output = initial_setup_output[0][1]
spectra_scan_type = initial_setup_output[1][0]
areas_scan_type = initial_setup_output[1][1]
spectra_check_string = initial_setup_output[1][2]
print(f"Copy input files to /{file_path_input}")

Select the ionization mode
(a) +EI
(b) +CI NH3
(c) +CI CH4
(d) -CI NH3
(e) -CI CH4
Your choice:  a


Select the scan mode
(a) Scan
(b) SIM
Your choice:  b




Copy input files to /Input_Data


In [4]:
'''
Function for retrieving csv file and returning in indicated format
'''
def get_csv(return_type, title, initialdir, skip_blank_lines, skiprows):
    import tkinter as tk
    from tkinter import filedialog
    import GeneralFunctions as GeneralFunctions
    csvfile = tk.filedialog.askopenfilename(title=title, initialdir=initialdir, filetype=(("CSV files", ".csv"), ("all files", "*.*")))
    file = open(csvfile)
    if return_type == 'lines':
        file_lines = GeneralFunctions.get_file_lines(file)
        file.close()
        return(file_lines)
    elif return_type == 'df' :
        file_df = pd.read_csv(file, skip_blank_lines=skip_blank_lines, skiprows=skiprows)
        file.close()
        return(file_df)
    elif return_type == 'both':
        file_df = pd.read_csv(file, skip_blank_lines=skip_blank_lines, skiprows=skiprows)
        file.close()
        file = open(csvfile)
        file_lines = GeneralFunctions.get_file_lines(file)
        file.close()
        return(file_lines, file_df)
print("defined function for retrieving csv files")

defined function for retrieving csv files


In [5]:
'''
Fetch profile in both formats
'''
# profile_lines, profile_df = get_csv(return_type='both', title='Profile', initialdir=file_path_input)
profile_df = get_csv(return_type='df', title='Profile', initialdir=file_path_input, skip_blank_lines=False, skiprows=None)
# can change the return type to lines once I add legacy functionality back in
print("profile file imported")

defined function for exporting aligned peak table as csv
profile file imported


In [6]:
'''
Count the number of features in the profile dataframe
'''
max_feature_count = max(profile_df.count())
min_feature_count = min(profile_df.count())
print(f"{max_feature_count} total features\n {max_feature_count-min_feature_count} feature(s) missing profile data")

137 total features
 0 feature(s) missing profile data


In [7]:
'''
Format and Display the first 5 lines of the profile dataframe
'''
# profile_df['RT'] = profile_df['RT'].astype(float)
# profile_df['Start'] = profile_df['Start'].astype(float)
# profile_df['End'] = profile_df['End'].astype(float)
# profile_df['#Carbons'] = profile_df['#Carbons'].astype(int)
# profile_df['Quasi Mol ion'] = profile_df['Quasi Mol ion'].astype(float)
# profile_df['MW'] = profile_df['MW'].astype(float)
# profile_df['Quasi Mol ion'] = profile_df['Quasi Mol ion'].apply(np.floor)
# profile_df['MW'] = profile_df['MW'].apply(np.floor)
def rename_duplicates(input_series):
    renamed_series = input_series
    series_list = []
    for item in input_series:
        counter = 1
        new_item = item
        while new_item in series_list:
            counter += 1
            new_item = item + '_' + str(counter)
        series_list.append(new_item)
    renamed_series_values = pd.Series(series_list)
    return renamed_series_values

profile_df['Peak'] = rename_duplicates(profile_df['Peak'])
profile_df.head()

,Peak,RT,Start,End,Name,MW,#Carbons,Quasi Mol ion,Formula,RawMW,Exact Mass
0,RT_4.16,4.16,4.100,4.220,RT_4.16,1,1,1,C1H4,1,1
1,RT_4.34,4.34,4.280,4.400,RT_4.34,1,1,1,C1H4,1,1
2,RT_4.53,4.53,4.495,4.565,RT_4.53,1,1,1,C1H4,1,1
3,RT_4.68,4.68,4.630,4.730,RT_4.68,1,1,1,C1H4,1,1
4,RT_4.84,4.84,4.800,4.880,RT_4.84,1,1,1,C1H4,1,1


In [8]:
profile_df['Peak']

0       RT_4.16
1       RT_4.34
2       RT_4.53
3       RT_4.68
4       RT_4.84
         ...   
132    RT_33.09
133    RT_33.66
134    RT_35.01
135    RT_35.34
136    RT_37.36
Name: Peak, Length: 137, dtype: object

In [9]:
def get_areas_df(file_path_input):
    print("fetching areas")
    areas_df = get_csv(return_type='df', title='Areas', initialdir=file_path_input, skip_blank_lines=True, skiprows=1)
    areas_df.columns.values[0] = 'SampleID'
    areas_df.head()
    return areas_df

In [10]:
def get_sampleID_from_area(sample_file):
    ## works to return sample ID from full info line
    ## #"+CI Scan (rt: 4.762 min) 210505_NH3-pos-split-scan_361-R2.D  Subtract "
    start_character = "\\"
    end_character = ".D"
    start_index = sample_file.rfind(start_character) + len(start_character)
    end_index = sample_file.rfind(end_character)
    sampleID = sample_file[start_index:end_index]
    return sampleID

In [11]:
def get_area_tables(profile_df, areas_df):
    print("Separating features by sample")
    areas_df_split = [y for x, y in areas_df.groupby('SampleID', as_index=False)] # Split dataframe by SampleID

    profile_headers = profile_df.columns.tolist()
    extra_headers = ['sampleID', 'in_profile', 'flag','RT_difference']
    area_headers = areas_df.columns.tolist()
    feature_headers = extra_headers + area_headers
    columnheaders = profile_headers + extra_headers + area_headers
    all_rows = [columnheaders]

    simplified_headers = profile_headers
    all_simplified_features = []

    first_run_for_sample = True
    for index1,profilefeature in profile_df.iterrows():
        profileID, profileRT, lowerrt, upperrt = profilefeature['Peak'], profilefeature['RT'], profilefeature['Start'], profilefeature['End']
#         print(f"Working on: {profileID}")
        profile_line = profilefeature.tolist()
        this_simplified_feature = profilefeature.tolist()
        this_feature = []
        flag, previous_sample, first_run_for_feature = None, None, True
        sample_counter = 0 
        for sample_df in areas_df_split:
            sample_df['RT'] = pd.to_numeric(sample_df['RT'], errors='coerce') # Sets non numeric as NaN
            sample_df = sample_df[sample_df['RT'] > 0]
            if len(sample_df) > 0: # Only process data files and lines with at least 1 features
                sample_file = sample_df['SampleID'].iloc[0]
                sampleID = get_sampleID_from_area(sample_file)                 
#                 print(f"processing: {sampleID}")
                if first_run_for_sample:
                    simplified_headers.append(sampleID)
                in_profile=False
                possible_features = []

    #                 rt_index = sample_df.columns.get_loc("RT")
    #                 area_index = sample_df.columns.get_loc("Area")
                for index1,feature in sample_df.iterrows():
                    this_feature = []
                    rt,area = feature['RT'],feature['Area']
                    if lowerrt < rt < upperrt: # feature rt is within profile rt bounds
                        # feature confirmed in profile
                        flag = None
                        in_profile = True
                        RT_difference = rt - profileRT
                        RT_difference = round(RT_difference,4)
                        this_feature=[sampleID, in_profile, flag, RT_difference]
                        feature_values = feature.values.tolist()
                        this_feature = this_feature + feature_values
                        possible_features.append(this_feature)

                if in_profile:
                    sample_counter += 1
                    possible_features_df = pd.DataFrame(possible_features,columns=feature_headers)
                    this_feature_df = possible_features_df[possible_features_df['RT_difference'] == possible_features_df['RT_difference'].min()]
                    this_feature_df = this_feature_df.loc[0]
                    area = this_feature_df['Area']
                    this_feature = this_feature_df.tolist()
                    this_row = profile_line + this_feature
                    all_rows.append(this_row)
                    this_simplified_feature.append(area)
    #                 print(f"{profileID} added for {sampleID}")
                else:
                    this_simplified_feature.append(None)
    #                 print(f"{profileID} not found for {sampleID}")
        all_simplified_features.append(this_simplified_feature)
        first_run_for_sample = False
        print(f"{profileID} found in {sample_counter} samples.")
            
    #         print(f"{profileID} completed")
#     simplified_area_table = simplified_headers + all_simplified_features
    detailed_areas_df = pd.DataFrame(all_rows,columns=columnheaders)
    simplified_areas_df = pd.DataFrame(all_simplified_features,columns=simplified_headers)
    return(detailed_areas_df, simplified_areas_df)




In [12]:

def ask_areas():
    '''ask if user wants to get area data'''
    from tkinter.messagebox import askyesno
    title = 'Process peak area data?'
    message = 'Would you like to process peak area data?'
    get_areas = askyesno(title=title, message=message)
    return get_areas

def process_areas(file_path_input, file_path_output):
    get_areas = ask_areas()
    if get_areas is True:
        areas_df = get_areas_df(file_path_input)
        print("Processing areas data table...")
        detailed_areas_df, simplified_areas_df = get_area_tables(profile_df, areas_df)
        print("Dataframe for all features with details complete")
        output_file_name1 = 'detailed_areas_df.csv'
        output_file_name2 = 'simplified_areas_df.csv'
        print(f"Exporting {output_file_name1} and {output_file_name2} to {file_path_output}")
        output_file_name_path1 = os.path.join(file_path_output, output_file_name1)
        output_file_name_path2 = os.path.join(file_path_output, output_file_name2)
        try:
            detailed_areas_df.to_csv(output_file_name_path1, sep=',', na_rep='', index=False, header=False)
            print("Export complete")
        except:
            print(f"Error in exporting {output_file_name1} to {output_file_name_path1}.\nMake sure a file with this name is not already open.\nContinuing without exporting this file.")
        try:
            simplified_areas_df.to_csv(output_file_name_path2, sep=',', na_rep='', index=False, header=True)
            print("Export complete")
        except:
            print(f"Error in exporting {output_file_name2} to {output_file_name_path2}.\nMake sure a file with this name is not already open.\nContinuing without exporting this file.")

        simplified_areas_df.head()
    else:
        quit()
        
process_areas(file_path_input, file_path_output)
    

fetching areas
Processing areas data table...
Separating features by sample
RT_4.16 found in 2 samples.
RT_4.34 found in 2 samples.
RT_4.53 found in 1 samples.
RT_4.68 found in 2 samples.
RT_4.84 found in 2 samples.
RT_5.17 found in 2 samples.
RT_5.27 found in 1 samples.
RT_5.54 found in 1 samples.
RT_5.79 found in 1 samples.
RT_6.27 found in 2 samples.
RT_6.59 found in 2 samples.
RT_6.91 found in 2 samples.
RT_6.98 found in 2 samples.
RT_7.26 found in 2 samples.
RT_7.42 found in 1 samples.
RT_7.57 found in 2 samples.
RT_7.74 found in 1 samples.
RT_7.84 found in 2 samples.
RT_7.97 found in 1 samples.
RT_8.09 found in 1 samples.
RT_8.22 found in 2 samples.
RT_8.48 found in 2 samples.
RT_8.6 found in 2 samples.
RT_8.66 found in 2 samples.
RT_8.77 found in 2 samples.
RT_8.99 found in 2 samples.
RT_9.16 found in 2 samples.
RT_9.57 found in 2 samples.
RT_9.67 found in 1 samples.
RT_9.79 found in 2 samples.
RT_10.38 found in 2 samples.
RT_10.53 found in 2 samples.
RT_10.9 found in 1 samples.

In [13]:
quit()